# Column Transformer

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(r"C:\Users\prajwal\Downloads\covid.csv")

In [3]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [4]:
df['cough'].value_counts()

cough
Mild      62
Strong    38
Name: count, dtype: int64

In [5]:
df['city'].value_counts()

city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

In [6]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [7]:
# Gender and city are nomila categorical data
# cough is ordinal categorical data
# age and fever are numerical data

In [8]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [9]:
# We will implementing One Hot Encoding in gender
# cough -> ordinal Encoder
# city also -> one hot encoder

In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],
                                                test_size=0.2)

In [11]:
X_train

,age,gender,fever,cough,city
28,16,Male,104.0,Mild,Kolkata
46,19,Female,101.0,Mild,Mumbai
12,25,Female,99.0,Strong,Kolkata
6,14,Male,101.0,Strong,Bangalore
20,12,Male,98.0,Strong,Bangalore
...,...,...,...,...,...
4,65,Female,101.0,Mild,Mumbai
0,60,Male,103.0,Mild,Kolkata
68,54,Female,104.0,Strong,Kolkata
49,44,Male,104.0,Mild,Mumbai


**Without Col-Transformer**

In [12]:
# adding simple imputer to fever col
# Filling fever values
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])
# missing values will be replaced by mean values.

# also the test data
X_test_fever = si.fit_transform(X_test[['fever']])
                                 
X_train_fever.shape

(80, 1)

In [13]:
# Ordinalencoding -> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
# Specify-> Mild will get lower value than stronger.
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

In [14]:
# OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first',sparse_output=False)
# multi_collinearity problem solve by [drop='first'] and [sparse_output = False] will solve converting to np.array prblm
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

(80, 4)

In [15]:
# Now we will be concatinating the whole data

In [16]:
# Extracting Age -> by dropping all the cols except age.
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [17]:
# Producing whole data by concatinating values to age.
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

**Using Column Transformer**

In [18]:
from sklearn.compose import ColumnTransformer

In [19]:
transformer = ColumnTransformer(
    # will take 2 parameters -> transformers _list and remainders.
    # pas_through -> leave remaining cols, whereas drop-> drop the cols.
    transformers=[
        # transfermers are always pass in tuple.
        # Passing three tuples here.
        # name tnf1, tnf2,... are pass.
        # Then col_name is pass inside the list where we want to perform the specific operation.
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
         # We are implementing Mild and Strong categories to 'cough' col
    ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [20]:
transformer.fit_transform(X_train).shape

(80, 7)

In [21]:
transformer.transform(X_test).shape

(20, 7)